In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [101]:
data = pd.read_csv('sp500_data.csv')

data['sector'] = data['sector'].astype('category').cat.codes

data.dropna(subset=['result'], inplace=True)

bins = [-10, -.5, -.25, 0, .25, .5, 10]
data['result_bin'] = pd.cut(data['result'], bins).astype('category').cat.codes

pct_change = data['result']
#data = data.drop(['result'], axis=1)

X = data[data.columns[2:-1]]
y = data['result_bin']

train_idx = data[data['year'] < 2020].index.values
test_idx = data[data['year']==2020].index.values
X_train = X.loc[train_idx]
X_test = X.loc[test_idx]
y_train = y.loc[train_idx]
y_test = y.loc[test_idx]

IQR = X_train.quantile(.75) - X_train.quantile(.25)
lower = X_train.quantile(.25) - 1.5 * IQR
upper = X_train.quantile(.75) + 1.5 * IQR

for i in X_train.columns:
    # set outliers to the upper/lower limit
    X_train.loc[X_train[i]<lower[i], i] = lower[i]
    X_train.loc[X_train[i]>upper[i], i] = upper[i]

    X_test.loc[X_test[i]<lower[i], i] = lower[i]
    X_test.loc[X_test[i]>upper[i], i] = upper[i]

    # fill nan values with the median value for that column
    X_test[i].fillna(X_train[i].median(), inplace=True)
    X_train[i].fillna(X_train[i].median(), inplace=True)

    # normalize the data
    X_test[i] = (X_test[i] - X_train[i].min()) / (X_train[i].max() - X_train[i].min())
    X_train[i] = (X_train[i] - X_train[i].min()) / (X_train[i].max() - X_train[i].min())

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1267, 19) (476, 19) (1267,) (476,)


C:\Users\Bogdan\AppData\Local\Temp\ipykernel_8164\163926654.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-4.5' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  X_train.loc[X_train[i]<lower[i], i] = lower[i]
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_8164\163926654.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-4.5' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  X_test.loc[X_test[i]<lower[i], i] = lower[i]
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_8164\163926654.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [106]:
rows = data[data.ticker == 'NVDA']
rows[['year', 'gross_margin', 'result']]

,year,gross_margin,result
1364,2018,1.086782,-0.426009
1365,2019,0.577672,0.771121
1366,2020,0.656805,1.252548
1367,2021,0.349145,0.828534


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
model = LogisticRegression(max_iter=1000)
model = model.fit(X_train, y_train)
preds = model.predict(X_test)

print(accuracy_score(preds, y_test))

0.24789915966386555


In [62]:
display(pd.concat([X_test.reset_index(), pd.Series(preds)], axis=1))

,index,sector,current_ratio,quick_ratio,cash_ratio,operating_cashflow,equity_ratio,debt_to_equity,debt_to_assets,interest_coverage,asset_turnover,receivables_turnover,gross_margin,profit_margin,operating_margin,return_on_equity,return_on_assets,eps,p/e,0
0,3,0.7,0.497558,0.542397,0.292060,0.587874,0.474693,0.506624,0.529537,0.196922,0.564840,0.370802,0.490376,0.508755,0.539455,0.845113,0.725785,0.417259,0.666402,3
1,7,0.7,0.596629,0.592155,0.563445,0.413139,0.665101,0.372072,0.341381,0.000000,0.604256,0.279138,0.447800,0.432130,0.461703,0.518126,0.608709,1.000000,0.353127,3
2,11,0.6,0.605979,0.606926,0.588246,0.506407,0.566076,0.424184,0.437585,0.631945,0.298804,0.378554,0.471073,0.292221,0.326312,0.293460,0.320714,0.472270,0.466477,2
3,14,0.6,0.852374,0.750517,0.187649,0.606703,0.115647,0.000000,0.888380,0.533286,0.339459,0.485936,0.543026,0.542122,0.645598,0.000000,0.594441,1.000000,0.353613,3
4,18,0.6,1.000000,1.000000,1.000000,1.000000,1.000000,0.289660,0.003744,0.539761,0.531229,0.533740,0.678822,0.756739,0.596737,0.649272,1.000000,0.470716,0.508854,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,1930,0.2,0.264686,0.224795,0.212955,0.606012,0.000000,0.110638,1.000000,0.621459,0.722587,0.522818,0.499147,0.631893,0.709136,0.052369,1.000000,0.374799,0.826062,3
472,1935,0.9,0.284062,0.345814,0.032019,0.442284,0.375611,0.692478,0.629578,0.650995,0.695525,0.493610,0.419404,0.276495,0.329796,0.300201,0.313123,1.000000,0.424750,3
473,1938,0.6,0.367286,0.470155,0.096350,0.434776,0.654532,0.376572,0.351730,0.566269,0.184659,0.294581,0.716522,0.524776,0.498212,0.396932,0.452835,0.335228,0.000000,4
474,1943,0.5,0.008031,0.006815,0.015596,0.240799,0.296039,1.000000,0.701089,0.539761,0.016083,0.358797,0.904146,0.663802,0.827263,0.385866,0.329622,0.559335,0.378616,2


In [74]:
test_data = data.loc[test_idx, data.columns]

In [80]:
result = pd.concat([test_data.reset_index(), pd.Series(preds)], axis=1)

In [89]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(result[['ticker', 'year', 'result_bin', 0]])

    ticker  year  result_bin  0
0      MMM  2020           3  3
1      AOS  2020           5  3
2      ABT  2020           4  2
3     ABBV  2020           4  3
4     ABMD  2020           5  5
5      ACN  2020           4  3
6     ATVI  2020           1  2
7      ADM  2020           4  3
8     ADBE  2020           4  3
9      ADP  2020           4  2
10     AES  2020           3  2
11     AFL  2020           4  2
12       A  2020           4  3
13     AIG  2020           5  2
14     APD  2020           2  2
15    AKAM  2020           3  2
16     ALK  2020           2  3
17     ALB  2020           5  2
18     ARE  2020           4  3
19    ALGN  2020           3  3
20    ALLE  2020           3  3
21     LNT  2020           3  3
22     ALL  2020           3  2
23   GOOGL  2020           5  2
24    GOOG  2020           5  2
25      MO  2020           3  4
26    AMZN  2020           3  2
27    AMCR  2020           3  2
28     AMD  2020           5  3
29     AEE  2020           3  3
30     A

In [95]:
filtered_df = result[result.iloc[:, -1] >= 4]
filtered_df = filtered_df[['ticker', 'year', 'result_bin', 0]]
print(filtered_df)

    ticker  year  result_bin  0
4     ABMD  2020           5  5
25      MO  2020           3  4
47    ANET  2020           5  5
49       T  2020           2  4
56     BAC  2020           4  4
82     CCL  2020           2  4
92    CERN  2020           3  4
97     CVX  2020           4  4
106    CFG  2020           4  4
109    CME  2020           4  4
115    CMA  2020           5  4
117    COP  2020           5  4
119    STZ  2020           3  4
121   CPRT  2020           5  4
129    DHI  2020           3  5
138   FANG  2020           5  4
140    DFS  2020           4  4
144    DIS  2020           4  4
157    ECL  2020           3  4
164    EOG  2020           5  4
171   ETSY  2020           3  5
176   EXPE  2020           4  4
184   FITB  2020           5  4
197    BEN  2020           5  5
200   GRMN  2020           3  5
206    GPC  2020           4  4
208     GL  2020           2  4
222    HLT  2020           4  4
227    HST  2020           3  5
232   HBAN  2020           3  4
237   IN

In [100]:
last_column = filtered_df.iloc[:, -1]
second_to_last_column = filtered_df.iloc[:, -2]

returns = 1
bins = [-5, -.25, -.125, 0, .125, .25, 5]
for i in second_to_last_column:
    returns *= (1+bins[i])
print(returns)

42.239075297304424
